In [100]:
import pandas as pd
import numpy as np

# Dataset

This is a dataset similar to the one which Lighter Capital has sent over. I will walk through how to format the data accordingly (so that we can actually use it to calculate metrics). THESE ARE RANDOM VALUES.

In [101]:
df = pd.DataFrame()

columns = ['Income|Revenue', 'Income|Interest', 'Other Income|Benefits', 
           'Other Income|Insurance', 'Expenses|Internet', 'Expenses|Marketing']

values = np.random.rand(25, len(columns)).reshape(-1, 1) * 100

df['Account'] = columns * 25
df['Balance'] = values

df.head()

,Account,Balance
0,Income|Revenue,11.928220
1,Income|Interest,72.524515
2,Other Income|Benefits,28.274257
3,Other Income|Insurance,77.729235
4,Expenses|Internet,50.004480


The dataset is completely unusable. So, to make it better. Lets first split up the "account" column into the main cateogry (income/other income/expenses) and sub-category (revenue/interest). We will make it two seperate columns and drop the original account column (don't need it anymore).

In [102]:
df[['Main', 'Sub']] = df['Account'].str.split("|", expand=True)
df = df.drop(['Account'], axis = 1)
df.head()

,Balance,Main,Sub
0,11.928220,Income,Revenue
1,72.524515,Income,Interest
2,28.274257,Other Income,Benefits
3,77.729235,Other Income,Insurance
4,50.004480,Expenses,Internet


With this, we can already gain more information when compared to the previous dataset. For example, we can get the mean balance across all three main categories (income/other income/expenses) and that can be used to calculate profit (income - expenses).

In [103]:
groups = df.groupby('Main')['Balance'].mean()

print("GroupBy Balance\n")
print(groups)

print("\n Potential Profit\n")

profit = groups['Income'] - groups['Expenses']

print(f'${profit}')

GroupBy Balance

Main
Expenses        53.608531
Income          51.351330
Other Income    49.837844
Name: Balance, dtype: float64

 Potential Profit

$-2.2572010579752373


However, we can potentially do better. Pandas has this thing called multi-hierarchy which allows us to represent the "groupby" above as columns. The syntax for creating a multihierarchy is shown below.

In [104]:
df = df.sort_values(by=['Main', 'Sub'], ascending=False)
df = df.set_index(['Main', 'Sub'])

df.loc[df.index.get_level_values(0).unique().tolist()]

Balance
Main         Sub                 
Other Income Insurance  77.729235
             Insurance   1.105609
             Insurance  86.147098
             Insurance  85.542745
             Insurance  35.548542
...                           ...
Expenses     Internet   74.207877
             Internet   14.739378
             Internet   54.684394
             Internet   20.057720
             Internet   58.087635

[150 rows x 1 columns]

Now, calculating profit becomes much much easier. 

In [105]:
df.loc['Income'] - df.loc['Expenses']

,Balance
Sub,
Interest,NaN
Interest,NaN
Interest,NaN
Interest,NaN
Interest,NaN
...,...
Revenue,NaN
Revenue,NaN
Revenue,NaN
